<a href="https://colab.research.google.com/github/hhkbhamza/Rag-Workshop/blob/main/Copy_of_Headstarter_RAG_Workshop_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![Img](https://app.theheadstarter.com/static/hs-logo-opengraph.png)

# Headstarter RAG Workshop

**Skills: HuggingFace, LangChain, Pinecone**

**Other Resources:**
- [Get your Groq API Key](https://console.groq.com/keys)
- [Get your Pinecone API Key](https://www.pinecone.io/)


### What is RAG anyway?


![withoutRAG](https://github.com/user-attachments/assets/649d6101-b63a-4750-997a-b6abc25e5609)

![withRAG](https://github.com/user-attachments/assets/e6dd9c46-0bf9-4c31-bd72-a27939ef82b8)

Retrieval-Augmented Generation (RAG) is a technique primarily used in GenAI applications to improve the quality and accuracy of generated text by LLMs by combining two key processes: retrieval and generation.

### Breaking It Down:
#### Retrieval:

- Before generating a response, the system first looks up relevant information from a large database or knowledge base. This is like searching through a library or the internet to find the most useful facts, articles, or data related to the question or topic.

#### Generation:

- Once the relevant information is retrieved, the system then uses it to help generate a response. This is where the model, like GPT, creates new text (answers, explanations, etc.) based on the retrieved information.

# Install libraries

In [ ]:
! pip install langchain langchain-community openai groq tiktoken pinecone-client langchain_pinecone unstructured pdfminer==20191125 pdfminer.six==20221105 pillow_heif unstructured_inference sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 37.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 39.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.1 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of pdfplumber to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━

In [ ]:
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader, WebBaseLoader, YoutubeLoader, DirectoryLoader, TextLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sklearn.metrics.pairwise import cosine_similarity
from langchain_pinecone import PineconeVectorStore
from langchain.embeddings import OpenAIEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
from google.colab import userdata
from langchain.schema import Document
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone
from openai import OpenAI
import numpy as np
import tiktoken
import os
from groq import Groq

pinecone_api_key = userdata.get("PINECONE_API_KEY")
os.environ['PINECONE_API_KEY'] = pinecone_api_key

openai_api_key = userdata.get("OPENAI_API_KEY")
os.environ['OPENAI_API_KEY'] = openai_api_key

groq_api_key = userdata.get("GROQ_API_KEY")
os.environ['GROQ_API_KEY'] = groq_api_key

# Initialize the HuggingFace Embeddings client

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

<ipython-input-6-83794808db26>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or d

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
text = "Hello my name is Amir"

query_result = embeddings.embed_query(text)

In [ ]:
len(query_result)

384

# Initialize the Groq client

In [ ]:
# Free Llama 3.1 API via Groq

groq_client = Groq(api_key=os.getenv('GROQ_API_KEY'))

# Calculating sentence similarity with embeddings

In [ ]:
def get_huggingface_embeddings(text, model_name="sentence-transformers/all-MiniLM-L6-v2"):
    model = SentenceTransformer(model_name)
    return model.encode(text)


def cosine_similarity_between_sentences(sentence1, sentence2):
    # Get embeddings for both sentences
    embedding1 = np.array(get_huggingface_embeddings(sentence1))
    embedding2 = np.array(get_huggingface_embeddings(sentence2))

    # Reshape embeddings for cosine_similarity function
    embedding1 = embedding1.reshape(1, -1)
    embedding2 = embedding2.reshape(1, -1)

    print("Embedding for Sentence 1:", embedding1)
    print("\nEmbedding for Sentence 2:", embedding2)

    # Calculate cosine similarity
    similarity = cosine_similarity(embedding1, embedding2)
    return similarity[0][0]


# Example usage
sentence1 = "I like running to the park"
sentence2 = "I like running to the office"


similarity = cosine_similarity_between_sentences(sentence1, sentence2)
print(f"\n\nCosine similarity between '{sentence1}' and '{sentence2}': {similarity:.4f}")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Embedding for Sentence 1: [[ 1.23337246e-02 -5.13279764e-03  3.57208848e-02  3.30768302e-02
   7.82396719e-02 -4.72629350e-03  6.77640066e-02 -1.38818845e-02
   5.46722263e-02  3.72701623e-02 -6.69813827e-02 -1.25908284e-02
  -3.56482230e-02  1.04401549e-02  7.53818303e-02  2.34690451e-04
   5.66774085e-02  5.25491722e-02 -3.24604735e-02  5.10555133e-02
  -7.36738369e-02  2.87814029e-02  4.96469587e-02  1.52121764e-02
  -1.48359686e-01  9.21717212e-02 -3.28955650e-02 -1.19502367e-02
  -1.94677711e-02 -1.07078562e-02 -7.19258264e-02 -2.60948092e-02
   3.90276383e-03  2.24410407e-02 -2.83260699e-02  3.94298211e-02
   3.03058382e-02 -6.47537112e-02 -2.83420784e-03  6.29675165e-02
   5.27709723e-03 -2.52472498e-02  6.72674701e-02 -6.20138133e-04
  -2.60539651e-02 -1.77838989e-02 -1.57480724e-02 -1.01696076e-02
   1.10002562e-01 -2.35369720e-04  1.14311509e-01 -1.48504437e-03
   1.67892501e-02 -2.43191961e-02  3.09247011e-03  3.71284708e-02
  -8.57886225e-02  4.56232540e-02  3.15336883e-02 

# Load in the Data

Learn more about the dataset [here](https://www.kaggle.com/datasets/ayoubcherguelaine/company-documents-dataset)

In [ ]:
! kaggle datasets download -d ayoubcherguelaine/company-documents-dataset
! unzip company-documents-dataset.zip

Dataset URL: https://www.kaggle.com/datasets/ayoubcherguelaine/company-documents-dataset
License(s): apache-2.0
  0% 0.00/9.34M [00:00<?, ?B/s]
100% 9.34M/9.34M [00:00<00:00, 184MB/s]
Archive:  company-documents-dataset.zip
  inflating: CompanyDocuments/Inventory Report/monthly-Category/monthly-Category/StockReport_2016-07_1.pdf  
  inflating: CompanyDocuments/Inventory Report/monthly-Category/monthly-Category/StockReport_2016-07_2.pdf  
  inflating: CompanyDocuments/Inventory Report/monthly-Category/monthly-Category/StockReport_2016-07_3.pdf  
  inflating: CompanyDocuments/Inventory Report/monthly-Category/monthly-Category/StockReport_2016-07_4.pdf  
  inflating: CompanyDocuments/Inventory Report/monthly-Category/monthly-Category/StockReport_2016-07_5.pdf  
  inflating: CompanyDocuments/Inventory Report/monthly-Category/monthly-Category/StockReport_2016-07_6.pdf  
  inflating: CompanyDocuments/Inventory Report/monthly-Category/monthly-Category/StockReport_2016-07_7.pdf  
  inflating: 

In [ ]:
def process_directory(directory_path):
    data = []
    for root, _, files in os.walk(directory_path):
        for file in files:

            file_path = os.path.join(root, file)
            print(f"Processing file: {file_path}")
            loader = PyPDFLoader(file_path)
            data.append({"File": file_path, "Data": loader.load()})

    return data

directory_path = "/content/CompanyDocuments"
documents = process_directory(directory_path)


Processing file: /content/CompanyDocuments/Shipping orders/order_10491.pdf
Processing file: /content/CompanyDocuments/Shipping orders/order_10921.pdf
Processing file: /content/CompanyDocuments/Shipping orders/order_10504.pdf
Processing file: /content/CompanyDocuments/Shipping orders/order_10332.pdf
Processing file: /content/CompanyDocuments/Shipping orders/order_10673.pdf
Processing file: /content/CompanyDocuments/Shipping orders/order_10783.pdf
Processing file: /content/CompanyDocuments/Shipping orders/order_10928.pdf
Processing file: /content/CompanyDocuments/Shipping orders/order_10804.pdf
Processing file: /content/CompanyDocuments/Shipping orders/order_10644.pdf
Processing file: /content/CompanyDocuments/Shipping orders/order_10636.pdf
Processing file: /content/CompanyDocuments/Shipping orders/order_10999.pdf
Processing file: /content/CompanyDocuments/Shipping orders/order_10476.pdf
Processing file: /content/CompanyDocuments/Shipping orders/order_10991.pdf
Processing file: /content

In [ ]:
documents

[{'File': '/content/CompanyDocuments/Shipping orders/order_10491.pdf',
  'Data': [Document(metadata={'source': '/content/CompanyDocuments/Shipping orders/order_10491.pdf', 'page': 0}, page_content='Order ID: 10491\nShipping Details:\nShip Name: Furia Bacalhau e Frutos do Mar\nShip Address: Jardim das rosas n. 32\nShip City: Lisboa\nShip Region: Southern Europe\nShip Postal Code: 1675\nShip Country: Portugal\nCustomer Details:\nCustomer ID: FURIB\nCustomer Name: Furia Bacalhau e Frutos do Mar\nEmployee Details:\nEmployee Name: Laura Callahan\nShipper Details:\nShipper ID: 3\nShipper Name: Federal Shipping\nOrder Details:\nOrder Date: 2017-03-31\nShipped Date: 2017-04-08\nProducts:\n--------------------------------------------------------------------------------------------------\nProduct: Gula Malacca\nQuantity: 15\nUnit Price: 15.5\nTotal: 232.5\n--------------------------------------------------------------------------------------------------\nProduct: Original Frankfurter grüne Soße\

# Initialize Pinecone

In [ ]:
# Make sure to create a Pinecone index with 384 dimensions

index_name = "rag-workshop"

namespace = "company-documents"

vectorstore = PineconeVectorStore(index_name=index_name, embedding=embeddings)

In [ ]:
document_data = []
for document in documents:

    document_source = document['Data'][0].metadata['source']
    document_content = document['Data'][0].page_content

    file_name = document_source.split("/")[-1]
    folder_names = document_source.split("/")[2:-1]

    doc = Document(
        page_content = f"<Source>\n{document_source}\n</Source>\n\n<Content>\n{document_content}\n</Content>",
        metadata = {
            "file_name": file_name,
            "parent_folder": folder_names[-1],
            "folder_names": folder_names
        }
    )
    document_data.append(doc)

In [ ]:
document_data

[Document(metadata={'file_name': 'order_10491.pdf', 'parent_folder': 'Shipping orders', 'folder_names': ['CompanyDocuments', 'Shipping orders']}, page_content='<Source>\n/content/CompanyDocuments/Shipping orders/order_10491.pdf\n</Source>\n\n<Content>\nOrder ID: 10491\nShipping Details:\nShip Name: Furia Bacalhau e Frutos do Mar\nShip Address: Jardim das rosas n. 32\nShip City: Lisboa\nShip Region: Southern Europe\nShip Postal Code: 1675\nShip Country: Portugal\nCustomer Details:\nCustomer ID: FURIB\nCustomer Name: Furia Bacalhau e Frutos do Mar\nEmployee Details:\nEmployee Name: Laura Callahan\nShipper Details:\nShipper ID: 3\nShipper Name: Federal Shipping\nOrder Details:\nOrder Date: 2017-03-31\nShipped Date: 2017-04-08\nProducts:\n--------------------------------------------------------------------------------------------------\nProduct: Gula Malacca\nQuantity: 15\nUnit Price: 15.5\nTotal: 232.5\n--------------------------------------------------------------------------------------

In [ ]:
vectorstore_from_documents = PineconeVectorStore.from_documents(
    document_data,
    embeddings,
    index_name=index_name,
    namespace=namespace
)

# Insert data into Pinecone

# Perform RAG

In [ ]:
pc = Pinecone(api_key=userdata.get("PINECONE_API_KEY"))

pinecone_index = pc.Index(index_name)

In [ ]:
query = "What are some common products bought by Mary Saveley"

In [ ]:
raw_query_embedding = get_huggingface_embeddings(query)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
raw_query_embedding

array([-4.26725596e-02, -6.77617192e-02,  1.35143986e-02, -9.34239384e-03,
       -3.53645310e-02,  1.50140822e-01,  2.53909770e-02, -9.55799315e-03,
       -3.04681305e-02, -8.55066180e-02,  7.04338029e-02, -3.25601851e-03,
        1.76474210e-02, -6.33681267e-02, -1.97862964e-02,  5.99328689e-02,
       -3.36771784e-03,  1.15839437e-01, -2.16768794e-02, -2.27578115e-02,
       -4.50396240e-02,  2.58584227e-02,  4.44791801e-02,  8.40743557e-02,
        2.40690890e-03, -1.55011723e-02,  6.56717038e-03,  2.42126826e-02,
       -4.94253030e-03, -8.99974853e-02, -2.78666709e-02,  1.11753186e-02,
       -3.28647792e-02,  2.51273136e-03,  1.29573699e-02,  3.35337520e-02,
        4.09512520e-02,  3.65742072e-02,  4.78686020e-02, -4.06755414e-03,
       -3.53386849e-02, -1.24937393e-01, -4.32422245e-03, -5.11763711e-03,
       -7.36645460e-02,  2.81967465e-02,  6.48829807e-03,  9.16031599e-02,
        2.23698728e-02, -4.23583426e-02, -6.91724867e-02,  7.57981511e-03,
       -3.12628709e-02, -

In [ ]:
top_matches = pinecone_index.query(vector=raw_query_embedding.tolist(), top_k=10, include_metadata=True, namespace=namespace)

In [ ]:
top_matches

{'matches': [{'id': '2dc4c8d1-0a87-490e-a853-24cb558beec8',
              'metadata': {'file_name': 'purchase_orders_10478.pdf',
                           'folder_names': ['CompanyDocuments',
                                            'PurchaseOrders'],
                           'parent_folder': 'PurchaseOrders',
                           'text': '<Source>\n'
                                   '/content/CompanyDocuments/PurchaseOrders/purchase_orders_10478.pdf\n'
                                   '</Source>\n'
                                   '\n'
                                   '<Content>\n'
                                   'Purchase Orders\n'
                                   'Order ID Order Date Customer Name\n'
                                   '10478 2017-03-18 Mary Saveley\n'
                                   'Products\n'
                                   'Product ID: Product: Quantity: Unit '
                                   'Price:\n'
                         

In [ ]:
contexts = [item['metadata']['text'] for item in top_matches['matches']]

In [ ]:
contexts

['<Source>\n/content/CompanyDocuments/PurchaseOrders/purchase_orders_10478.pdf\n</Source>\n\n<Content>\nPurchase Orders\nOrder ID Order Date Customer Name\n10478 2017-03-18 Mary Saveley\nProducts\nProduct ID: Product: Quantity: Unit Price:\n10 Ikura 20 24.8\nPage 1\n</Content>',
 '<Source>\n/content/CompanyDocuments/PurchaseOrders/purchase_orders_10806.pdf\n</Source>\n\n<Content>\nPurchase Orders\nOrder ID Order Date Customer Name\n10806 2017-12-31 Mary Saveley\nProducts\nProduct ID: Product: Quantity: Unit Price:\n2 Chang 20 19\n65 Louisiana Fiery Hot Pepper Sauce 2 21.05\n74 Longlife Tofu 15 10\nPage 1\n</Content>',
 '<Source>\n/content/CompanyDocuments/PurchaseOrders/purchase_orders_10334.pdf\n</Source>\n\n<Content>\nPurchase Orders\nOrder ID Order Date Customer Name\n10334 2016-10-21 Mary Saveley\nProducts\nProduct ID: Product: Quantity: Unit Price:\n52 Filo Mix 8 5.6\n68 Scottish Longbreads 10 10\nPage 1\n</Content>',
 '<Source>\n/content/CompanyDocuments/PurchaseOrders/purchase_o

In [ ]:
augmented_query = "<CONTEXT>\n" + "\n\n-----\n\n".join(contexts[:10]) + "\n-----\n</CONTEXT>\n\n\n\nMY QUERY: \n"+query


In [ ]:
augmented_query

"<CONTEXT>\n<Source>\n/content/CompanyDocuments/PurchaseOrders/purchase_orders_10478.pdf\n</Source>\n\n<Content>\nPurchase Orders\nOrder ID Order Date Customer Name\n10478 2017-03-18 Mary Saveley\nProducts\nProduct ID: Product: Quantity: Unit Price:\n10 Ikura 20 24.8\nPage 1\n</Content>\n\n-----\n\n<Source>\n/content/CompanyDocuments/PurchaseOrders/purchase_orders_10806.pdf\n</Source>\n\n<Content>\nPurchase Orders\nOrder ID Order Date Customer Name\n10806 2017-12-31 Mary Saveley\nProducts\nProduct ID: Product: Quantity: Unit Price:\n2 Chang 20 19\n65 Louisiana Fiery Hot Pepper Sauce 2 21.05\n74 Longlife Tofu 15 10\nPage 1\n</Content>\n\n-----\n\n<Source>\n/content/CompanyDocuments/PurchaseOrders/purchase_orders_10334.pdf\n</Source>\n\n<Content>\nPurchase Orders\nOrder ID Order Date Customer Name\n10334 2016-10-21 Mary Saveley\nProducts\nProduct ID: Product: Quantity: Unit Price:\n52 Filo Mix 8 5.6\n68 Scottish Longbreads 10 10\nPage 1\n</Content>\n\n-----\n\n<Source>\n/content/CompanyD

In [ ]:
system_prompt = f"""Tour are an expert at understanding and analyzing company data - particularly shipping orders, purchase orders, invoices and inventory reports.

Answer any questions I have, based on the data provided. Always consider all of the context provided when forming a response.
"""

llm_response = groq_client.chat.completions.create(
    model="llama-3.1-70b-versatile",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": augmented_query}
    ]
)

response = llm_response.choices[0].message.content
print(response)


Based on the provided purchase order data, we can see that Mary Saveley has purchased the following products on multiple occasions:

1. Ikura (Product ID: 10): 
   - Purchase Order 10478 (2017-03-18) - 20 units
   - Purchase Order 10450 (2017-02-19) - 20 units

2. Uncle Bob's Organic Dried Pears (Product ID: 7): 
   - Purchase Order 10459 (2017-02-27) - 16 units (Invoice 10459 confirms the purchase)
   - Purchase Order 10546 (2017-05-23) - 10 units

3. Louisiana Fiery Hot Pepper Sauce (Product ID: 65): 
   - Purchase Order 10251 (2016-07-08) - 20 units
   - Purchase Order 10806 (2017-12-31) - 2 units


# Putting it all together